In [1]:
import os
import gc

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split

from torch_geometric.nn import global_mean_pool
from torch_geometric.loader import DataLoader
from torch_geometric.data import Batch
from torch_geometric.nn import GCNConv, VGAE

import os
import kagglehub
from kagglehub import KaggleDatasetAdapter

import pandas as pd

from tqdm import tqdm
from tqdm.contrib import tmap
from tqdm.contrib.concurrent import process_map

from torchvision import transforms

from concurrent.futures import ProcessPoolExecutor

from lib.lib import SiameseSignatureDataset

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np

from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

# Data Preparation

## prepare data from mallapraveen/signature-matching
## and construct it using data.csv

In [12]:
df = pd.read_csv('data.csv')

def dataset_path():
    path = kagglehub.dataset_download("mallapraveen/signature-matching")
    return os.path.join(path, 'custom\\full')

def transform(**kwargs):
    return transforms.Compose([
        transforms.Grayscale(num_output_channels=kwargs['num_output_channels']),
        transforms.Resize(kwargs['resize']),
        transforms.ToTensor(),
    ])
    
dataset = SiameseSignatureDataset(
    root_dir=dataset_path(),
    signer_folders=df,
    transform=transform(num_output_channels=1, resize=(32, 32)
))

Loaded 85246 signature images (genuine + forged)


## split the data
### train dataset & validation dataset

In [13]:
total_size = len(dataset)
train_size = int(0.8 * total_size)
val_size = total_size - train_size
train_dataset, val_dataset = random_split(
    dataset,
    [train_size, val_size],
    generator=torch.Generator().manual_seed(42)
)
print(f"Dataset sizes - Train: {train_size}, Validation: {val_size}")

Dataset sizes - Train: 68196, Validation: 17050


In [14]:
train_dataset[0]

(Data(x=[1024, 3], edge_index=[2, 3968]),
 Data(x=[1024, 3], edge_index=[2, 3968]),
 0)

## load the data using dataloader

In [15]:
train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=4
)

val_loader = DataLoader(
    val_dataset,
    batch_size=256,
    shuffle=False,
    num_workers=4
)

In [16]:
next(iter(train_loader))

[DataBatch(x=[32768, 3], edge_index=[2, 126976], batch=[32768], ptr=[33]),
 DataBatch(x=[32768, 3], edge_index=[2, 126976], batch=[32768], ptr=[33]),
 tensor([1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0,
         0, 0, 0, 1, 0, 0, 1, 0])]

# Model Preparation

In [17]:
class GNNEncoder(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, latent_dim):
        super(GNNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv_mu = GCNConv(hidden_channels, latent_dim)
        self.conv_logvar = GCNConv(hidden_channels, latent_dim)

    def forward(self, x, edge_index):
        # Step 1: Aggregate node features from neighbors
        x = F.relu(self.conv1(x, edge_index))

        # Step 2: Output mean and log variance
        mu = self.conv_mu(x, edge_index)
        logvar = self.conv_logvar(x, edge_index)

        return mu, logvar

In [73]:
class SiameseNetwork(nn.Module):
    def __init__(self, fe_model, latent_dim):
        super(SiameseNetwork, self).__init__()
        self.encoder = fe_model
        self.embedding_dim = latent_dim
        
        self.projector = nn.Sequential(
            nn.Linear(self.embedding_dim, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),

            nn.Linear(128, 64),
            nn.ReLU(inplace=True),

            nn.Linear(64, 2)
        )

    def forward_once(self, x, edge_index, batch):
        mu, _ = self.encoder(x, edge_index)
        graph_emb = global_mean_pool(mu, batch) 
        # x = torch.flatten(x, 1)
        # return x
        return graph_emb

    def forward(self, x1, x2,
               edge_index1, edge_index2,
               batch):
        emb1 = self.forward_once(x1, edge_index1, batch)
        emb2 = self.forward_once(x2, edge_index2, batch)

        # Combine embeddings (abs difference works well for verification)
        combined = torch.abs(emb1 - emb2)

        # Predict same/forged
        out = self.projector(combined)
        return out

# Hyperparameters

In [74]:
w_d = 1e-5
epochs = 50
learning_rate = 1e-3
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

# Training Preparation

In [75]:
img1, _, _ = next(iter(train_loader))

input_dim = img1.x.shape[1]
hidden_dim = 64
latent_dim = 128

In [76]:
# Load your trained GNN-VAE
checkpoint = torch.load('VGAE_Model.pt', map_location=device)
vgae = VGAE(GNNEncoder(in_channels=input_dim, hidden_channels=hidden_dim, latent_dim=latent_dim)).to(device)
vgae.load_state_dict(checkpoint)
vgae.eval()

VGAE(
  (encoder): GNNEncoder(
    (conv1): GCNConv(3, 64)
    (conv_mu): GCNConv(64, 128)
    (conv_logvar): GCNConv(64, 128)
  )
  (decoder): InnerProductDecoder()
)

In [77]:
model = SiameseNetwork(vgae, latent_dim=128).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

## train steps

In [78]:
def train_step(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss, correct, total = 0.0, 0, 0

    for x1, x2, label in tqdm(dataloader, desc="Training", leave=False):
        x1, x2, label = x1.to(device), x2.to(device), label.to(device)

        # Forward
        output = model(x1.x.to(device),
                    x2.x.to(device),
                    x1.edge_index.to(device),
                    x2.edge_index.to(device),
                    x1.batch)  # logits shape: [batch, 2]

        print(output)
        
        loss = criterion(output, label)

        # Backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Metrics
        total_loss += loss.item() * x1.size(0)
        preds = torch.argmax(output, dim=1)
        correct += (preds == label).sum().item()
        total += label.size(0)

    avg_loss = total_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

## validation steps

In [79]:
def val_step(model, dataloader, criterion, device):
    model.eval()
    total_loss, correct, total = 0.0, 0, 0
    all_labels = []
    all_preds = []
    all_probs = []

    with torch.no_grad():
        for x1, x2, label in tqdm(dataloader, desc="Validating", leave=False):
            x1, x2, label = x1.to(device), x2.to(device), label.to(device)
            output = model(x1.x.to(device),
                    x2.x.to(device),
                    x1.edge_index.to(device),
                    x2.edge_index.to(device),
                    x1.batch)

            loss = criterion(output, label)
            total_loss += loss.item() * x1.size(0)

            probs = torch.softmax(output, dim=1)[:, 1]  # Probability of class 1 ("genuine")
            preds = torch.argmax(output, dim=1)

            all_labels.extend(label.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

            correct += (preds == label).sum().item()
            total += label.size(0)

    avg_loss = total_loss / total
    accuracy = correct / total
    return avg_loss, accuracy, np.array(all_labels), np.array(all_preds), np.array(all_probs)


# Training Phase

In [80]:
writer = SummaryWriter(log_dir="runs/siamese_signature_experiment")

best_auc = 0.0

for epoch in range(epochs):
    train_loss, train_acc = train_step(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc, y_true, y_pred, y_prob = val_step(model, val_loader, criterion, device)

    # --- Confusion Matrix ---
    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    fig_cm, ax_cm = plt.subplots(figsize=(4, 4))
    disp.plot(ax=ax_cm, cmap="Blues", colorbar=False)
    writer.add_figure("ConfusionMatrix/val", fig_cm, global_step=epoch)
    plt.close(fig_cm)

    # --- ROC Curve and AUC ---
    fpr, tpr, _ = roc_curve(y_true, y_prob)
    roc_auc = auc(fpr, tpr)
    fig_roc, ax_roc = plt.subplots()
    ax_roc.plot(fpr, tpr, color='blue', lw=2, label=f"AUC = {roc_auc:.3f}")
    ax_roc.plot([0, 1], [0, 1], color='gray', linestyle='--')
    ax_roc.set_xlabel("False Positive Rate")
    ax_roc.set_ylabel("True Positive Rate")
    ax_roc.legend(loc="lower right")
    writer.add_figure("ROC/val", fig_roc, global_step=epoch)
    plt.close(fig_roc)

    # --- Precision, Recall, F1 ---
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average="binary"
    )

    # Log scalar metrics
    writer.add_scalar("Loss/train", train_loss, epoch)
    writer.add_scalar("Loss/val", val_loss, epoch)
    writer.add_scalar("Accuracy/train", train_acc, epoch)
    writer.add_scalar("Accuracy/val", val_acc, epoch)
    writer.add_scalar("AUC/val", roc_auc, epoch)
    writer.add_scalar("Precision/val", precision, epoch)
    writer.add_scalar("Recall/val", recall, epoch)
    writer.add_scalar("F1/val", f1, epoch)

    print(f"Epoch [{epoch+1}/{epochs}] "
          f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} "
          f"| Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f} "
          f"| AUC: {roc_auc:.4f} | F1: {f1:.4f}")

    # --- Save best model by AUC ---
    if roc_auc > best_auc:
        best_auc = roc_auc
        torch.save(model.state_dict(), "best_siamese_signature.pth")

writer.close()


Training:   0%|                                                                    | 1/2132 [00:22<13:08:45, 22.21s/it]

tensor([[-0.0717,  0.0536],
        [-0.0675,  0.0492],
        [-0.0857,  0.0579],
        [-0.0815,  0.0552],
        [-0.0734,  0.0619],
        [-0.0684,  0.0462],
        [-0.0760,  0.0601],
        [-0.0856,  0.0561],
        [-0.0746,  0.0558],
        [-0.0804,  0.0523],
        [-0.0642,  0.0634],
        [-0.0599,  0.0670],
        [-0.0648,  0.0626],
        [-0.0850,  0.0530],
        [-0.0612,  0.0661],
        [-0.0723,  0.0476],
        [-0.0750,  0.0463],
        [-0.0908,  0.0584],
        [-0.0791,  0.0569],
        [-0.0736,  0.0497],
        [-0.0593,  0.0593],
        [-0.0798,  0.0512],
        [-0.0657,  0.0462],
        [-0.0831,  0.0482],
        [-0.0763,  0.0576],
        [-0.0766,  0.0608],
        [-0.0692,  0.0648],
        [-0.0645,  0.0522],
        [-0.0642,  0.0443],
        [-0.0708,  0.0518],
        [-0.0845,  0.0677],
        [-0.0738,  0.0581]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-0.0664,  0.0611],
        [-0.0859,  0.0604],
    

Training:   0%|▎                                                                      | 9/2132 [00:22<53:49,  1.52s/it]

tensor([[-0.0815,  0.0559],
        [-0.0892,  0.0567],
        [-0.0849,  0.0582],
        [-0.0794,  0.0624],
        [-0.0775,  0.0471],
        [-0.0813,  0.0652],
        [-0.0731,  0.0534],
        [-0.0799,  0.0536],
        [-0.0753,  0.0591],
        [-0.0690,  0.0580],
        [-0.0686,  0.0491],
        [-0.0770,  0.0617],
        [-0.0698,  0.0562],
        [-0.0868,  0.0577],
        [-0.0804,  0.0668],
        [-0.0712,  0.0578],
        [-0.0799,  0.0559],
        [-0.0883,  0.0603],
        [-0.0736,  0.0686],
        [-0.0809,  0.0628],
        [-0.0806,  0.0699],
        [-0.0830,  0.0605],
        [-0.0670,  0.0573],
        [-0.0676,  0.0347],
        [-0.0884,  0.0538],
        [-0.0802,  0.0603],
        [-0.0725,  0.0627],
        [-0.0797,  0.0697],
        [-0.0841,  0.0688],
        [-0.0781,  0.0680],
        [-0.0746,  0.0462],
        [-0.0830,  0.0619]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-0.0783,  0.0630],
        [-0.0934,  0.0679],
    

Training:   1%|▌                                                                     | 17/2132 [00:22<19:13,  1.83it/s]

...

Training:   7%|████▉                                                                | 154/2132 [00:28<01:29, 22.18it/s]

tensor([[-0.0186,  0.0275],
        [-0.0702,  0.0620],
        [-0.0151,  0.0161],
        [-0.0336,  0.0332],
        [-0.0543,  0.0248],
        [ 0.1838, -0.1030],
        [ 0.0885, -0.0481],
        [ 0.1052, -0.0703],
        [-0.0375,  0.0451],
        [-0.0685,  0.0514],
        [-0.0502,  0.0376],
        [ 0.1327, -0.0846],
        [-0.0425,  0.0098],
        [-0.0037, -0.0178],
        [-0.0514,  0.0427],
        [-0.0361,  0.0394],
        [-0.0296,  0.0264],
        [-0.0703,  0.0711],
        [-0.0150,  0.0285],
        [-0.0372,  0.0224],
        [ 0.0419, -0.0573],
        [ 0.0171,  0.0028],
        [-0.0532,  0.0487],
        [-0.0354,  0.0097],
        [ 0.1726, -0.1009],
        [ 0.0516, -0.0269],
        [ 0.1260, -0.0758],
        [ 0.1414, -0.1063],
        [-0.0288,  0.0221],
        [-0.0690,  0.0674],
        [ 0.0597, -0.0305],
        [-0.0668,  0.0406]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[ 0.0111, -0.0132],
        [-0.0421,  0.0357],
    

Training:   7%|█████                                                                | 158/2132 [00:28<01:28, 22.32it/s]

tensor([[-0.0366,  0.0240],
        [ 0.1225, -0.0698],
        [ 0.0662, -0.0677],
        [-0.0619,  0.0547],
        [-0.0511,  0.0422],
        [-0.0032, -0.0083],
        [-0.0783,  0.0595],
        [-0.0528,  0.0378],
        [ 0.0670, -0.0731],
        [ 0.1773, -0.1436],
        [-0.0546,  0.0473],
        [-0.0343,  0.0221],
        [ 0.0280, -0.0333],
        [ 0.1379, -0.0544],
        [ 0.1133, -0.1058],
        [ 0.0328, -0.0043],
        [-0.0674,  0.0598],
        [ 0.1530, -0.1440],
        [ 0.1973, -0.1137],
        [ 0.0707, -0.0577],
        [-0.0720,  0.0447],
        [-0.0549,  0.0462],
        [ 0.0600, -0.0509],
        [ 0.0630, -0.0283],
        [-0.0751,  0.0501],
        [ 0.0034, -0.0036],
        [-0.0713,  0.0507],
        [ 0.0147,  0.0090],
        [ 0.0707, -0.0560],
        [ 0.2112, -0.0824],
        [ 0.2346, -0.1280],
        [-0.0444,  0.0383]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-0.0327,  0.0191],
        [-0.0736,  0.0591],
    

KeyboardInterrupt: 